In [10]:
import os
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [11]:
from scripts.utils import preprocess

file_paths = ['../data/hourly/ada_lunarcrush_timeseries_hourly.csv', '../data/hourly/btc_lunarcrush_timeseries_hourly.csv', '../data/hourly/doge_lunarcrush_timeseries_hourly.csv', '../data/hourly/eth_lunarcrush_timeseries_hourly.csv', '../data/hourly/xmr_lunarcrush_timeseries_hourly.csv', '../data/hourly/xrp_lunarcrush_timeseries_hourly.csv', '../data/hourly/aave_lunarcrash_timeseries_hourly.csv']

crypto_dfs = {}
for path in file_paths:
    crypto = os.path.basename(path).split('_')[0]
    if os.path.exists(path):  # Check if the file exists
        crypto_dfs[crypto] = preprocess(path)
    else:
        print(f"File does not exist: {path}") 

# Model Training on Financial-only data

In [12]:
from scripts.utils import simplify_df

for crypto, df in crypto_dfs.items():
    crypto_dfs[crypto] = simplify_df(df)

## Baseline

In [25]:
def baseline_model(df):
    df['predicted_close'] = df['close'].shift(1)
    df = df.dropna(subset=['predicted_close'])
    mae = mean_absolute_error(df['close'], df['predicted_close'])
    return mae

In [5]:
baseline_results = {}

for crypto, df in crypto_dfs.items():
    mae = baseline_model(df)
    baseline_results[crypto] = mae

# Output the baseline MAE results
print("Baseline MAE results for each cryptocurrency:")
for crypto, mae in baseline_results.items():
    print(f"{crypto}: {mae:.4f}")

Baseline MAE results for each cryptocurrency:
ada: 0.0047
btc: 121.2149
doge: 0.0008
eth: 8.7742
xmr: 1.0392
xrp: 0.0036
aave: 17.2299


## LSTM

In [26]:
import tensorflow as tf
from tensorflow.keras import layers as tfkl
from tensorflow.keras.models import Model as tfkModel

def build_simple_LSTM_regressor(input_shape, output_units=1):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # LSTM layer
    lstm = tfkl.LSTM(16, activation='leaky_relu', return_sequences=True)(input_layer)
    lstm = tfkl.LSTM(16, activation='leaky_relu')(lstm)

    # Output layer for regression
    output_layer = tfkl.Dense(output_units)(lstm)  # Single unit for regression output

    # Connect input and output through the Model class
    model = tfkModel(inputs=input_layer, outputs=output_layer, name='Simple_LSTM_regressor')

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    # Return the model
    return model

In [27]:
def fit(X_train, y_train, X_val, y_val):

    # Assuming input_shape is (5, n_features)
    input_shape = (X_train.shape[1], X_train.shape[2])
    output_units = 1

    # Build and compile the model
    model = build_simple_LSTM_regressor(input_shape, output_units)
    model.summary()

    # Train the model
    history = model.fit(
        x=X_train,
        y=y_train,
        batch_size=64,
        validation_data=(X_val, y_val),
        epochs=100,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
        ]
    ).history

    return model, history

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def evaluate(model, X_test, y_test):

    model.evaluate(X_test, y_test, verbose=1)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return mse, mae, mape

In [29]:
from scripts.utils import split_data_frame, apply_functions

def run_model_for_dataframe(df):
    # Split the DataFrame into train, validation, and test sets
    train, val, test = split_data_frame(df, 0.7, 0.2)

    # Apply any additional functions to preprocess the data
    X_train, y_train, X_val, y_val, X_test, y_test = apply_functions(train, test, val)

    # Fit the model
    model, history = fit(X_train, y_train, X_val, y_val)

    # Get the evaluation metrics for each set
    mse, mae, mape = evaluate(model, X_test, y_test)

    # Return the evaluation metrics along with the model and history
    return model, history, (mse, mae, mape)

# Assuming crypto_dfs is your dictionary of DataFrames for each crypto
summary_table = []

for crypto, df in crypto_dfs.items():
    print(f"Running model for {crypto}...")
    model, history, metrics = run_model_for_dataframe(df)

    # Unpack the metrics
    lstm = 'LSTM'
    mse, mae, mape = metrics

    # Append the metrics to the summary table
    summary_table.append({
        'Data': crypto,
        'Model': lstm,
        'MSE': mse,
        'MAE': mae,
        'MAPE': mape
    })

# Convert the summary table to a DataFrame
lstm_df = pd.DataFrame(summary_table)
lstm_df.to_csv('../results/lstm.csv')
lstm_df

Running model for ada...
Model: "Simple_LSTM_regressor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 5, 12)]           0         
                                                                 
 lstm_2 (LSTM)               (None, 5, 16)             1856      
                                                                 
 lstm_3 (LSTM)               (None, 16)                2112      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3985 (15.57 KB)
Trainable params: 3985 (15.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
423/423 [==============================] - 2s 2ms/step - loss: 0.0790 - mae: 0.1203 - val_loss: 0.0024 - v

,Data,Model,MSE,MAE,MAPE
0,ada,LSTM,0.018989,0.121723,0.494236
1,btc,LSTM,0.004930,0.058811,0.382770
2,doge,LSTM,0.000859,0.023134,2.298499
3,eth,LSTM,0.002221,0.039746,1.327664
4,xmr,LSTM,0.000498,0.018086,0.518633
5,xrp,LSTM,0.000541,0.018221,0.880469
6,aave,LSTM,0.307624,0.540885,0.780253


## LSTM-CNN

In [30]:
import tensorflow as tf
from tensorflow.keras import layers as tfkl
from tensorflow.keras.models import Model as tfkModel

def build_improved_LSTM_CNN(input_shape, output_units):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # CNN Layer 1
    cnn = tfkl.Conv1D(256, 3, padding='same')(input_layer)  # Increased number of filters and adjusted kernel size
    cnn = tfkl.LeakyReLU(alpha=0.2)(cnn)
    cnn = tfkl.MaxPooling1D(pool_size=2)(cnn)  # Added pool_size

    # CNN Layer 2
    cnn = tfkl.Conv1D(128, 3, padding='same')(cnn)  # Adjusted number of filters and kernel size
    cnn = tfkl.LeakyReLU(alpha=0.2)(cnn)
    cnn = tfkl.MaxPooling1D(pool_size=2)(cnn)  # Added pool_size

    # LSTM Layer 1
    lstm = tfkl.LSTM(128, return_sequences=True)(cnn)  # Increased the number of units
    lstm = tfkl.Dropout(0.2)(lstm)  # Added dropout for regularization

    # LSTM Layer 2
    lstm = tfkl.LSTM(128)(lstm)  # Adjusted the number of units
    lstm = tfkl.Dropout(0.2)(lstm)  # Added dropout for regularization

    # Feature Extractor Layer
    dense = tfkl.Dense(64)(lstm)  # Adjusted the number of units
    dense = tfkl.LeakyReLU(alpha=0.2)(dense)
    dense = tfkl.Dropout(0.2)(dense)  # Added dropout for regularization

    # Output layer for regression
    output_layer = tfkl.Dense(output_units)(dense)  # No activation for regression output

    # Connect input and output through the Model class
    model = tfkModel(inputs=input_layer, outputs=output_layer, name='Improved_LSTM_CNN')

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

    return model

In [31]:
import tensorflow as tf

def fit(X_train, y_train, X_val, y_val):

    input_shape = (X_train.shape[1], X_train.shape[2])
    output_units = 1  # Assuming we want to predict one feature, e.g., close price for one day ahead

    # Build and compile the model
    model = build_improved_LSTM_CNN(input_shape, output_units)
    model.summary()

    # Train the model
    history = model.fit(
        x=X_train,
        y=y_train,
        batch_size=32,
        validation_data=(X_val, y_val),
        epochs=100,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
        ]
    ).history

    return model, history

In [32]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def evaluate(model, X_test, y_test):

    model.evaluate(X_test, y_test, verbose=1)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return mse, mae, mape

In [33]:
from scripts.utils import split_data_frame, apply_functions

def run_model_for_dataframe(df):
    # Split the DataFrame into train, validation, and test sets
    train, val, test = split_data_frame(df, 0.7, 0.2)

    # Apply any additional functions to preprocess the data
    X_train, y_train, X_val, y_val, X_test, y_test = apply_functions(train, test, val)

    # Fit the model
    model, history = fit(X_train, y_train, X_val, y_val)

    # Get the evaluation metrics for each set
    mse, mae, mape = evaluate(model, X_test, y_test)

    # Return the evaluation metrics along with the model and history
    return model, history, (mse, mae, mape)

# Assuming crypto_dfs is your dictionary of DataFrames for each crypto
summary_table = []

for crypto, df in crypto_dfs.items():
    print(f"Running model for {crypto}...")
    model, history, metrics = run_model_for_dataframe(df)

    # Unpack the metrics
    lstm_cnn = 'LSTM-CNN'
    mse, mae, mape = metrics

    # Append the metrics to the summary table
    summary_table.append({
        'Data': crypto,
        'Model': lstm_cnn,
        'MSE': mse,
        'MAE': mae,
        'MAPE': mape
    })

# Convert the summary table to a DataFrame
lstm_cnn_df = pd.DataFrame(summary_table)
lstm_cnn_df.to_csv('../results/lstm_cnn.csv')
lstm_cnn_df

Running model for ada...
Model: "Improved_LSTM_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 5, 12)]           0         
                                                                 
 conv1d (Conv1D)             (None, 5, 256)            9472      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 5, 256)            0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 2, 256)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 2, 128)            98432     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 2, 128)            0         
                        

,Data,Model,MSE,MAE,MAPE
0,ada,LSTM-CNN,0.010752,0.094137,0.299837
1,btc,LSTM-CNN,0.001655,0.035705,1.077848
2,doge,LSTM-CNN,0.004618,0.055242,6.120459
3,eth,LSTM-CNN,0.000452,0.016264,0.348036
4,xmr,LSTM-CNN,0.012357,0.094177,3.755125
5,xrp,LSTM-CNN,0.001085,0.026604,1.588341
6,aave,LSTM-CNN,0.111124,0.312760,0.439349


## TRANSFORMER

In [34]:
from keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0, epsilon=1e-6, attention_axes=None, kernel_size=1):
    """
    Creates a single transformer block.
    """
    x = layers.LayerNormalization(epsilon=epsilon)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout,
        attention_axes=attention_axes
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=epsilon)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=kernel_size, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=kernel_size)(x)
    return x + res

In [35]:
def build_transfromer(head_size, num_heads, ff_dim, num_trans_blocks, mlp_units, dropout=0, mlp_dropout=0, attention_axes=None, epsilon=1e-6, kernel_size=1):
    """
    Creates final model by building many transformer blocks.
    """
    n_timesteps, n_features, n_outputs = 5, 12, 1
    inputs = tf.keras.Input(shape=(n_timesteps, n_features))
    x = inputs
    for _ in range(num_trans_blocks):
        x = transformer_encoder(x, head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, dropout=dropout, attention_axes=attention_axes, kernel_size=kernel_size, epsilon=epsilon)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(n_outputs)(x)
    return tf.keras.Model(inputs, outputs)

In [36]:
import time

def fit_transformer(X_train, y_train, X_val, y_val):
    """
    Compiles and fits our transformer with the provided training and validation data.
    """
    transformer = build_transfromer(head_size=128, num_heads=4, ff_dim=2, num_trans_blocks=4, mlp_units=[256], mlp_dropout=0.10, dropout=0.10, attention_axes=1)    
    
    transformer.compile(
        loss="mse",
        optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),
        metrics=["mae", "mape"])

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]

    start = time.time()
    history = transformer.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        batch_size=64,
        epochs=100,
        verbose=1,
        callbacks=callbacks
    ).history

    print(f"Training completed in {time.time() - start:.2f} seconds")
    return transformer, history

In [37]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def evaluate(model, X_test, y_test):

    model.evaluate(X_test, y_test, verbose=1)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return mse, mae, mape

In [38]:
def run_transformer_for_dataframe(df):
    
    # Preprocessing steps
    train, val, test = split_data_frame(df, 0.7, 0.2)
    X_train, y_train, X_val, y_val, X_test, y_test = apply_functions(train, test, val)

    transformer, history = fit_transformer(X_train, y_train, X_val, y_val)

    # Evaluate the model
    mse, mae, mape = evaluate(model, X_test, y_test)

    # Return the model, history, and metrics
    return transformer, history, (mse, mae, mape)

In [39]:
summary_table = []

for crypto, df in crypto_dfs.items():
    
    print(f"Running model for {crypto}...")
    model, history, metrics = run_transformer_for_dataframe(df)

    # Unpack the metrics
    transformer = 'Transformer'
    mse, mae, mape = metrics

    # Append the metrics to the summary table
    summary_table.append({
        'Data': crypto,
        'Model': transformer,
        'MSE': mse,
        'MAE': mae,
        'MAPE': mape
    })

# Convert the summary table to a DataFrame
transformer_df = pd.DataFrame(summary_table)
transformer_df.to_csv('../results/transformer.csv')
transformer_df

Running model for ada...
Epoch 1/100
423/423 [==============================] - 10s 20ms/step - loss: 0.0238 - mae: 0.0931 - mape: 18.1522 - val_loss: 0.0577 - val_mae: 0.2300 - val_mape: 45.7766 - lr: 0.0010
Epoch 2/100
423/423 [==============================] - 9s 20ms/step - loss: 0.0058 - mae: 0.0568 - mape: 12.3554 - val_loss: 0.0399 - val_mae: 0.1859 - val_mape: 36.8241 - lr: 0.0010
Epoch 3/100
423/423 [==============================] - 8s 20ms/step - loss: 0.0042 - mae: 0.0479 - mape: 10.0046 - val_loss: 0.0537 - val_mae: 0.2197 - val_mape: 43.7317 - lr: 0.0010
Epoch 4/100
423/423 [==============================] - 8s 19ms/step - loss: 0.0037 - mae: 0.0447 - mape: 9.2815 - val_loss: 0.0531 - val_mae: 0.2201 - val_mape: 43.8272 - lr: 0.0010
Epoch 5/100
423/423 [==============================] - 8s 20ms/step - loss: 0.0031 - mae: 0.0407 - mape: 7.9881 - val_loss: 0.0324 - val_mae: 0.1698 - val_mape: 33.6986 - lr: 0.0010
Epoch 6/100
423/423 [==============================] - 8s 20m

,Data,Model,MSE,MAE,MAPE
0,ada,Transformer,0.478494,0.678623,1.920413
1,btc,Transformer,0.000354,0.016008,0.373019
2,doge,Transformer,0.001068,0.023030,0.874563
3,eth,Transformer,0.012137,0.090027,0.420364
4,xmr,Transformer,0.000591,0.016065,1.065858
5,xrp,Transformer,0.000162,0.008548,0.301665
6,aave,Transformer,0.015875,0.102130,0.161040


### Results

In [40]:
models = ['LSTM', 'LSTM-CNN', 'Transformer']
cryptos = ['ada', 'btc', 'doge', 'eth', 'xmr', 'xrp', 'aave']
file_paths = ['../results/lstm.csv', '../results/lstm_cnn.csv', '../results/transformer.csv']

results = pd.DataFrame(index=models, columns=cryptos)

# Loop through each file and each crypto to fill the DataFrame
for model, file_path in zip(models, file_paths):
    df = pd.read_csv(file_path)
    for crypto in cryptos:
        # Assuming the MAE column in each file is named 'test_mae'
        results.at[model, crypto] = df.loc[df['Data'] == crypto, 'MAE'].values[0]

results

,ada,btc,doge,eth,xmr,xrp,aave
LSTM,0.121723,0.058811,0.023134,0.039746,0.018086,0.018221,0.540885
LSTM-CNN,0.094137,0.035705,0.055242,0.016264,0.094177,0.026604,0.31276
Transformer,0.678623,0.016008,0.02303,0.090027,0.016065,0.008548,0.10213
